In [5]:
import json
from landsatxplore.api import API
import getpass
import pandas as pd
import geopandas as gpd
import ipywidgets as widgets
from IPython.display import display
import matplotlib.pyplot as plt
from hublib.ui import FileUpload
import ipywidgets as widgets
from IPython.display import display


In [6]:
import hublib.ui as ui
from hublib.ui import HideCodeButton, RunAllButton

In [7]:
RunAllButton()

Button(description='Run All Cells', style=ButtonStyle())

In [8]:
RunAllButton(label="Run Cells", icon='fa-play-circle-o', style='danger', tooltip='Run All Cells', hide=True)

Button(button_style='danger', description='Run Cells', icon='play-circle-o', style=ButtonStyle(), tooltip='Run…

In [9]:
HideCodeButton()

Button(description='Hide Code Cells', style=ButtonStyle())

<IPython.core.display.Javascript object>

In [10]:
# called when all files finish uploading
def done_cb(w, name):
    # Do something with the files here...
    # We just print their names
    print("%s uploaded" % name)
    
    # reset clears and re-enables the widget for more uploads
    # You may want to do this somewhere else, depending on your GUI
    w.reset()

In [11]:
# username_input = widgets.Text(description='Enter your Earth Explorer username: ')
# password_input = widgets.Password(description='Enter your Earth Explorer password: ')
username = input('Enter your Earth Explorer username: ')
password = getpass.getpass('Enter your Earth Explorer password: ')
bbox_file_input = widgets.FileUpload(description='Upload Bbox Shapefile')

Enter your Earth Explorer username: maggiechen259
Enter your Earth Explorer password: ········


In [12]:
# # Process uploaded bbox shapefile
# bbox_filename = input('Enter the name of the uploaded bounding box shapefile (with extension): ')
# bbox_file_path = f'bbox/{bbox_filename}'

In [13]:
# this will allow you to select a single file to be uploaded
f = FileUpload("Upload Area of Interest", 
               "This is a description that appears when the mouse hovers over the name.",
               dir='tmpdir',
               cb=done_cb,
               maxsize='20M')
f


In [14]:
# Create multiple checkboxes
check1 = ui.Checkbox('Landsat 5', desc='Select Landsat 5', value=False)
check2 = ui.Checkbox('Landsat 7', desc='Select Landsat 7', value=False)
check3 = ui.Checkbox('Landsat 8', desc='Select Landsat 8', value=False)
check4 = ui.Checkbox('Landsat 9', desc='Select Landsat 9', value=False)

In [15]:
checkboxes_horizontal = widgets.VBox([check1, check2, check3,check4])

# Display the checkboxes in a horizontal layout
display(checkboxes_horizontal)

In [16]:
e1 = ui.Number(
        name='Cloud Cover',
        description="Cloud Cover",
        units='%',
        min=0,
        max=500,
        value=0
)
e1

Number(children=(HTML(value='<div data-toggle="popover" title="Cloud Cover\nValues must be in units of percent…

In [17]:
style = ui.Dropdown(
            name='Composite Color',
            description="Choose the composite color",
            value='True Color',
            options={'True Color', 'False Color'},
#             cb=plot
        )
style

Dropdown(children=(HTML(value='<div data-toggle="popover" title="Choose the composite color" data-container="b…

In [18]:
#upload area of interest

bbox = gpd.read_file('bbox/test_bbox.shp')
bounds = bbox.bounds.iloc[0].to_list()

In [19]:
# Initialize a new API instance
api = API(username, password)

# Perform a request
response = api.request(endpoint="dataset-catalogs")
print(response)

# Search for Landsat TM scenes
scenes = api.search(
    dataset='landsat_ot_c2_l2',
    bbox = bounds,
    start_date='2022-10-01',
    end_date='2022-12-31',
    max_cloud_cover=5
)
print(f"{len(scenes)} scenes found.")

# Process the result
for scene in scenes:
    print(scene['acquisition_date'].strftime('%Y-%m-%d'))
    # Write scene footprints to disk
    fname = f"{scene['landsat_product_id']}.geojson"
    with open(fname, "w") as f:
        json.dump(scene['spatial_coverage'].__geo_interface__, f)

# log out
api.logout()



{'EE': 'EarthExplorer', 'GV': 'GloVis', 'HDDS': 'HDDS Explorer'}
3 scenes found.
2022-10-05
2022-10-04
2022-10-03


In [20]:
# Create a DataFrame from the scenes
df_scenes = pd.DataFrame(scenes)
df_scenes = df_scenes[['display_id','wrs_path', 'wrs_row','satellite','cloud_cover','acquisition_date']]
df_scenes.sort_values('acquisition_date', ascending=False, inplace=True)
df_scenes

,display_id,wrs_path,wrs_row,satellite,cloud_cover,acquisition_date
0,LC09_L2SP_009010_20221005_20230326_02_T1,9,10,9,4,2022-10-05
1,LC08_L2SP_010011_20221004_20221012_02_T1,10,11,8,3,2022-10-04
2,LC09_L2SP_011010_20221003_20230327_02_T1,11,10,9,1,2022-10-03


In [21]:
from landsatxplore.earthexplorer import EarthExplorer
import os
import time
import requests


# Initialize the API
ee = EarthExplorer(username, password)

# Select the first scene
ID = 'LC08_L2SP_010010_20221004_20221012_02_T1'
output_dir = './data'


# Get the Earth Explorer website URL for the scene
ee_scene_url = f'https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&scene_id={ID}&node=INVSVC'

# Open the scene URL in a web browser for manual download
print(f'You can manually download the scene from the following URL:')
print(ee_scene_url)

# Define the local file path
local_file_path = os.path.join(output_dir, f'{ID}.tar')

# Download the scene using requests
try:
    response = requests.get(ee_scene_url)
    if response.status_code == 200:
        with open(local_file_path, 'wb') as f:
            f.write(response.content)
        print(f'{ID} successfully downloaded.')
    else:
        print(f'{ID} download failed. HTTP status code: {response.status_code}')
except Exception as e:
    print(f'{ID} download failed. Error: {str(e)}')

ee.logout()


You can manually download the scene from the following URL:
https://earthexplorer.usgs.gov/order/process?dataset_name=LANDSAT_8_C1&scene_id=LC08_L2SP_010010_20221004_20221012_02_T1&node=INVSVC
LC08_L2SP_010010_20221004_20221012_02_T1 successfully downloaded.


In [22]:
import tarfile

ID = 'LC08_L2SP_010010_20221004_20221012_02_T1'
file_path = './data/{}.tar'.format(ID)

try:
    tar = tarfile.open(file_path)
    tar.extractall('./data/{}'.format(ID))
    tar.close()
    print('Extraction successful')
except Exception as e:
    print(f'Error: {e}')


Error: file could not be opened successfully


In [ ]:
# import tarfile

# # Extract files from tar archive
# tar = tarfile.open('./data/{}.tar'.format(ID))
# tar.extractall('./data/{}'.format(ID))
# tar.close()